In [2]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [6]:
pages={}

In [7]:
pages[0]="""$BAR BITES

NACHOS (Veg/Chicken)

Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream 

190/230

CHEESE MASALA PAPAD

Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese

160

H.O.C SIGNATURE MASALA PEANUTS

A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices 

160

SPICY TRICOLOUR CHAAT

Fresh Peas, Corn Kernels & Black Gram Tossed With Chillies & Sirka onions

180

CHATPATA CHANA CHAAT

A Delectable Blend Of Bengal Gram And A Tangy Mixture

160

TANDOORI CHICKEN CHAAT

Clay Oven Roasted Chicken Blended With Classic Spices

220

CHEESE DYNABITES

Classic Cheese Balls Served with New Orleans Dip

220

$SALADS

GREEN SALAD

Traditional Green Cuts With Mint Yoghurt Dip

170

HOC SALAD (Veg/Chicken)

Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing

180/210

WATERMELON & FETA

An All Time Classic - With Balsamic Glaze

210

CAESAR SALAD (Veg/Chicken)

Crunchy Ice Berg Lettuce, Parmesan With the Classic Caesar Dressing

190/230"""

In [9]:
for line in pages[0].split('\n\n'):
    print(line)

$BAR BITES
NACHOS (Veg/Chicken)
Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream 
190/230
CHEESE MASALA PAPAD
Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese
160
H.O.C SIGNATURE MASALA PEANUTS
A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices 
160
SPICY TRICOLOUR CHAAT
Fresh Peas, Corn Kernels & Black Gram Tossed With Chillies & Sirka onions
180
CHATPATA CHANA CHAAT
A Delectable Blend Of Bengal Gram And A Tangy Mixture
160
TANDOORI CHICKEN CHAAT
Clay Oven Roasted Chicken Blended With Classic Spices
220
CHEESE DYNABITES
Classic Cheese Balls Served with New Orleans Dip
220
$SALADS
GREEN SALAD
Traditional Green Cuts With Mint Yoghurt Dip
170
HOC SALAD (Veg/Chicken)
Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing
180/210
WATERMELON & FETA
An All Time Classic - With Balsamic Glaze
210
CAESAR SALAD (Veg/Chicken)
Crunchy Ice Berg Lettuce, Parmesan With the Classic Caesar Dressing
190/230


In [10]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [8]:
pages[1]="""$VEG STARTERS

CHEESE & CORN TRIANGLE (new)

Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried

220

CHAMPIGNONS FARCIS (new)

Sauteed Button Mushroom Stuffed With Cheese & Herbs

230

CHILLI CHEESE CIGARA (new)

NEW

Deep Fried Rolls With Cheese, Chill & Herbs

220

HOC CHIP & DIP BASKET

390

Combination Of Onion Rings, Jalapeno Cheese Balls, Greek Potato Wedges, Habanero Green Chili Fries

FRENCH FRIES

French Fries Tossed With Hoc Signature Cajun Spice Seasoning

190

ONION RINGS

Extra Crispy Onion Rings With A Spicy Salsa

190

MUSHROOM LOADED FRIES

Cheesy French Fries Topped With Mushrooms And Herbs

230

GARLIC BREAD

Garlic Butter Flavoured Bread Topped With Cheese

190

CHEESE CORN TIKKI

American Com & Cottage Cheese Patty Served with Mint Chutney

220

CRISPY CHILLY MUSHROOM

Crispy Fried Mushrooms Tossed With Oriental Spices

220

CRISPY CHILLY BABY CORN

Crispy Fried Babycom Tossed With Oriental Spices

190

CRISPY CORN CHILLY PEPPER

Fred Undi Lightly Crispy & Seasoned With Lime Juice, Salt, Chilli & Pepper

240

HONEY CHILLY POTATO

Crispy Fried Potatoes Tossed With Chilli And Honey

190"""

In [10]:
pages[2]="""$VEG STARTERS

SESAME CHILLI PANEER

Cottage Cheese Cubes Tossed In A Sweet & Spicy Sauce, Finished With Sesame Seeds

270

PANDAN LEAF COTTAGE CHEESE

Tender Cottage Cheese Wrapped In Fragrant Pandan Leaves, With Chilli Coriander Sauce

280

MEZZE BASKET

A Combination Of Hummus, Mutabal & Labneh, Served With Pita Bread

340

$FROM THE CLAY OVEN

MAKHMALI PANEER TIKKA

Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices

270

ANGARI PANEER TIKKA

Clay Oven Roasted Spicy Morsels Of Cottage Cheese

270

TANDOORI STUFFED MUSHROOMS

Mushrooms Filled With A Blend Of Indian Spices And Tangy Stuffing Mix, Adding A Unique Taste On The Bite

250

TANDOORI VEG MOMOS

Soft And Juicy Vegetable Momos Marinated With Tandoori Masala And Char Grilled

220

$CHICKEN STARTERS

CHICKEN LOADED FRIES

Cheesy French Fries Topped With Chicken & Herbs

260

SHANG DONG CHICKEN LOLLY POP

Crispy Lolly pops Crunched To Perfection & Tossed in our Signature Shang Dong Drip

260"""

In [12]:
pages[3]="""$CHICKEN STARTERS

HOC SIGNATURE CHICKEN WINGS

Sweet Chilly / Spiced Szechuan

260

PEPPER GARLIC CHICKEN

Cubes Of Chicken Tossed With Garlic And Pepper In Oriental Style

260

GUNTUR JUNCTION CHICKEN

Chicken Cubes Tossed With Freshly Ground Red Spices And Finished In South Indian Style

270

ANDHRA CHILLI CHICKEN *VERY HOT*

All Time Favourite Andhra Style Chicken Cooked With Coriander And Green Chilli Paste And Fresh Green Chillies

270

LEMON PEPPER CHICKEN

Slice Chicken Wok Tossed, Finished With Pepper Garlic & Lime

260

JIANG CHILLI CHICKEN

Chicken Tossed With Shredded Ginger And Chilli In Jiang Style

260

CHICKEN GHEE ROAST

Chicken Roasted With A Fiery Red, Tangy And Spicy With The Flavour Of Ghee

280

MASALA FRY CHICKEN (new)

Classic Indian Style With Bone Deep fried Chicken Served With Mint Chutney

250

$FROM THE CLAY OVEN

JAIPURI MURGH KA SOOLA

Succulent Pieces Of Chicken Zapped With Red Spices, Cinnamon Flavour & Finished In Tandoor

270

TANDOORI MURGH (HALF/FULL)

Chicken Marinated In Aromatic Indian Spices & Then Cooked In Tandoor

250/480

TANDOORI POMFRET

Indian Spices Marinated Whole Pomfret Skewered And Finished In Clay Oven

590"""

In [13]:
pages[4]="""$FROM THE CLAY OVEN

FISH TIKKA KALIMIRCH

Fresh Cubes Of Basa Fish Marinated With Crushed Black Pepper & Indian Spices, Finished In Clay Oven

320

AMRITSAR FISH TIKKA

Spicy Fish Tikka Marinated In Traditional Spices And Char Grilled

320

KALMI KEBAB (3pcs)

Clay Oven Roasted Chicken Drumsticks

260

ANGARI MURGH TIKKA

Clay Oven Roasted Spicy Morsels Of Chicken

260

TANDOORI LOLLIPOPS

Chicken Lollipops Marinated With Aromatic Indian Spices And Cooked In The Tandoor

270

MURGH MALAI TIKKA

Clay Oven Roasted Creamy Chunks Of Chicken

260

MURGH KASTURI KEBAB

Boneless Charcoal Smoked Chicken Tikka With Kasturi Methi Flavoured

260

TANDOORI CHICKEN MOMOS

Soft And Juicy Chicken Momos Marinated With Tandoori Masala And Char Grilled

260

TANDOORI KING PRAWNS

King Prawns Marinated With Yogurt, Lime Juice, Bhatti Spices And Slowly Cooked In Tandoor

499

$LAMB STARTERS

ANDHRA LAMB PEPPER FRY

Tender Mutton Pieces Tossed In South Indian Spices

350

MUTTON GHEE ROAST

Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee

370"""

In [15]:
pages[5]="""$LAMB STARTERS

LAHORI SEEKH KEBAB

Succulent Minced Lamb Kebabs In The True Lahori Street Food Style

370

RAILWAY MUTTON DRY

With Bone Mutton Cooked In Rich Indian Spices, As Served In The Railway Canteen Of The British Raj

350

$EGG STARTERS

EGG BHURJI

Desi Style Scrambled Eggs

170

SPICY MASALA OMELETTE

Slow Cooked Egg Omlet With Masala

180

EGG CHILLI 

Spicy Boiled Egg With Chilli Fry

210

$FROM THE GRILL

PERI PERI PANEER/CHICKEN

Perl Perl Basted Paneer / Chicken Cubes Served With Tzatziki Dip

260/280

THAI GRILLED FISH IN BANANA LEAF

Delicately Grilled Fish With Basil Flavour And Served Hot

330

HARISSA GRILLED FISH

Fish Marinated In A Spicy Tunisian Harissa Sauce And Grilled To Perfection

330

THAI GRILLED CHICKEN

Boneless Chicken Marinated With Thai Spices And Grilled

270"""

In [ ]:
pages[6]=

In [ ]:
pages[7]=

In [ ]:
pages[8]=

In [ ]:
pages[9]=

In [ ]:
pages[10]=

In [ ]:
pages[11]=